In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

dipolewarning = 0
warning = 0

##################################################################
######################## BEGIN USER INPUT ########################
##################################################################

# Choose atoms that give best representation of coordinate axes i.e. the most orthonormal (doesn't have to be exact).

# Atom positions to describe long and wide axes of molecule. Atom 1 will be for both the wide and long axes, 
# Atom 2 will be for the wide axis, and atom 3 will be for the long axis.

AtomGaussian1 = [11.551421, 10.160481, 5.027279]
AtomGaussian2 = [11.136760, 11.164395, 5.038672]
AtomGaussian3 = [9.256161, 9.233892, 5.019294]

# Atom positions (must be same atoms as above) from KRM calculations

AtomKRM1_m = [-6.251 , -5.602, -3.902]
AtomKRM2_m = [ -6.694 , -5.939, -4.835]
AtomKRM3_m = [-4.004 , -6.352, -4.642]

AtomKRM1_n = [-16.259, -0.094, -9.883]
AtomKRM2_n = [-15.573,  0.605, -9.414]
AtomKRM3_n = [-15.315,  0.619, -12.018]

# Dipole moment information from Gaussian Calculations

TransitionDipoleMoment_x_m = 11.33 * 2.54 # mutliplied by 2.54 to change Au to Debye
TransitionDipoleMoment_y_m = -5.578 * 2.54
TransitionDipoleMoment_z_m = 7.541 * 2.54

GroundStateDipoleMoment_x_m = 50.2194
GroundStateDipoleMoment_y_m = 45.8879
GroundStateDipoleMoment_z_m = 24.1195

ExcitedStateDipoleMoment_x_m = 50.2665
ExcitedStateDipoleMoment_y_m = 45.8418
ExcitedStateDipoleMoment_z_m = 24.1195

CenterOfMassFromKRM_m = [-4.87400, -5.74300, -3.73700]
CenterOfMassFromKRM_n = [-16.20400, -0.16500, -11.27800]

Permittivity = 1

UseCorrection = 1 # Set to 1 if you want initial coordinate guesses to be corrected.

##################################################################
######################### END USER INPUT #########################
##################################################################



### Finding dye separation vector ###

DyeSeparationVector = np.subtract(CenterOfMassFromKRM_m,CenterOfMassFromKRM_n)
DyeSeparationLength = np.linalg.norm(DyeSeparationVector)
print("Dye Separation =", DyeSeparationLength, "Angstroms")



## Correcting initial coordinate system for molecule n ###

if UseCorrection == 1:
    print("\n~~~~~ Corrections are being used for the initial coordinate axes guesses! ~~~~~")
    LongInitial_m = np.subtract(AtomGaussian1, AtomGaussian3)
    WideInitial_m = np.subtract(AtomGaussian1, AtomGaussian2)

    Correction_m = (-LongInitial_m[0]*WideInitial_m[0]-LongInitial_m[1]*WideInitial_m[1]-LongInitial_m[2]*WideInitial_m[2])/(LongInitial_m[0]+LongInitial_m[1]+LongInitial_m[2])

    NewAtom2_1_m = np.subtract(AtomGaussian2,Correction_m)
    NewAtom2_2_m = np.add(AtomGaussian2,Correction_m)

    NewWide1_m = np.subtract(AtomGaussian1,NewAtom2_1_m)
    NewWide2_m = np.subtract(AtomGaussian1,NewAtom2_2_m)

    TestDot1_m = np.dot(LongInitial_m,NewWide1_m)
    TestDot2_m = np.dot(LongInitial_m,NewWide2_m)

    if abs(TestDot1_m) > abs(TestDot2_m):
        AtomGaussian2 = NewAtom2_1_m

    if abs(TestDot1_m) < abs(TestDot2_m):
        AtomGaussian2 = NewAtom2_1_m

    
    
### Finding translation vector from the KRM calculations and applying to Gaussian structure ###

TranslationAtom1 = np.subtract(AtomKRM1_m,AtomKRM1_n)
TranslationAtom2 = np.subtract(AtomKRM2_m,AtomKRM2_n)
TranslationAtom3 = np.subtract(AtomKRM3_m,AtomKRM3_n)

AtomGaussian1Translated = np.add(AtomGaussian1, TranslationAtom1)
AtomGaussian2Translated = np.add(AtomGaussian2, TranslationAtom2)
AtomGaussian3Translated = np.add(AtomGaussian3, TranslationAtom3)

    
    
## Correcting initial coordinate system for molecule n ###
    
if UseCorrection == 1:
    LongInitial_n = np.subtract(AtomGaussian1Translated, AtomGaussian3Translated)
    WideInitial_n = np.subtract(AtomGaussian1Translated, AtomGaussian2Translated)

    Correction_n = (-LongInitial_n[0]*WideInitial_n[0]-LongInitial_n[1]*WideInitial_n[1]-LongInitial_n[2]*WideInitial_n[2])/(LongInitial_n[0]+LongInitial_n[1]+LongInitial_n[2])

    NewAtom2_1_n = np.subtract(AtomGaussian2Translated,Correction_n)
    NewAtom2_2_n = np.add(AtomGaussian2Translated,Correction_n)

    NewWide1_n = np.subtract(AtomGaussian1Translated,NewAtom2_1_n)
    NewWide2_n = np.subtract(AtomGaussian1Translated,NewAtom2_2_n)

    TestDot1_n = np.dot(LongInitial_n,NewWide1_n)
    TestDot2_n = np.dot(LongInitial_n,NewWide2_n)
    TestDot3_n = np.dot(LongInitial_n,WideInitial_n)

    if abs(TestDot1_n) > abs(TestDot2_n):
        AtomGaussian2Translated = NewAtom2_1_n
    if abs(TestDot1_n) < abs(TestDot2_n):
        AtomGaussian2Translated = NewAtom2_1_n

    
    
### Finding vectors describing coordinate system of molecule m and n from Gaussian calculation ###

LongAxis_m = np.subtract(AtomGaussian1,AtomGaussian3)
WideAxis_m = np.subtract(AtomGaussian1,AtomGaussian2)
VerticalAxis_m = np.cross(LongAxis_m,WideAxis_m)
LengthLongAxis_m = np.linalg.norm(LongAxis_m)
LenthWideAxis_m = np.linalg.norm(WideAxis_m)
LengthVerticalAxis_m = np.linalg.norm(VerticalAxis_m)
LongAxisUnit_m = LongAxis_m/LengthLongAxis_m
WideAxisUnit_m = WideAxis_m/LenthWideAxis_m
VerticalAxisUnit_m = VerticalAxis_m/LengthVerticalAxis_m

LongAxis_n = np.subtract(AtomGaussian1Translated,AtomGaussian3Translated)
WideAxis_n = np.subtract(AtomGaussian1Translated,AtomGaussian2Translated)
VerticalAxis_n = np.cross(LongAxis_n,WideAxis_n)
LengthLongAxis_n = np.linalg.norm(LongAxis_n)
LenthWideAxis_n = np.linalg.norm(WideAxis_n)
LengthVerticalAxis_n = np.linalg.norm(VerticalAxis_n)
LongAxisUnit_n = LongAxis_n/LengthLongAxis_n
WideAxisUnit_n = WideAxis_n/LenthWideAxis_n
VerticalAxisUnit_n = VerticalAxis_n/LengthVerticalAxis_n



### Check to make sure coordinate systems are set up correctly ###

LongAxis_mDotWideAxis_m = np.dot(LongAxis_m,WideAxis_m)
WideAxis_mDotVerticalAxis_m = np.dot(WideAxis_m,VerticalAxis_m)
VerticalAxis_mDotLongAxis_m = np.dot(VerticalAxis_m,LongAxis_m)

LongAxis_nDotWideAxis_n = np.dot(LongAxis_n,WideAxis_n)
WideAxis_nDotVerticalAxis_n = np.dot(WideAxis_n,VerticalAxis_n)
VerticalAxis_nDotLongAxis_n = np.dot(VerticalAxis_n,LongAxis_n)

if abs(LongAxis_mDotWideAxis_m) > .000001:
    print("\nWARNING! The long axis and wide axis for molecule m aren't orthogonal.")
    warning = warning + 1
if abs(WideAxis_mDotVerticalAxis_m) > .000001:
    print("WARNING! The wide axis and vertical axis for molecule m aren't orthogonal.")
    warning = warning + 1
if abs(VerticalAxis_mDotLongAxis_m) > .000001:
    print("WARNING! The vertical axis and long axis for molecule m aren't orthogonal.")
    warning = warning + 1
if abs(LongAxis_nDotWideAxis_n) > .000001:
    print("WARNING! The long axis and wide axis for molecule n aren't orthogonal.")
    warning = warning + 1
if abs(WideAxis_nDotVerticalAxis_n) > .000001:
    print("WARNING! The wide axis and vertical axis for molecule n aren't orthogonal.")
    warning = warning + 1
if abs(VerticalAxis_nDotLongAxis_n) > .000001:
    print("WARNING! The vertical axis and long axis for molecule n aren't orthogonal.")
    warning = warning + 1

    
    
### Finding projections of dipole moments onto molecule m's coordinates ###

GroundStateDipoleMoment_m = np.array([GroundStateDipoleMoment_x_m,GroundStateDipoleMoment_y_m,GroundStateDipoleMoment_z_m])
GroundStateDipoleMomentLength_m = np.linalg.norm(GroundStateDipoleMoment_m)
ExcitedStateDipoleMoment_m = np.array([ExcitedStateDipoleMoment_x_m,ExcitedStateDipoleMoment_y_m,ExcitedStateDipoleMoment_z_m])
ExcitedStateDipoleMomentLength_m = np.linalg.norm(ExcitedStateDipoleMoment_m)
TransitionDipoleMoment_m = np.array([TransitionDipoleMoment_x_m,TransitionDipoleMoment_y_m,TransitionDipoleMoment_z_m])
TransitionDipoleMomentLength_m = np.linalg.norm(TransitionDipoleMoment_m)

GroundStateDipoleMomentProjection_x_m = np.dot(LongAxisUnit_m,GroundStateDipoleMoment_m)
GroundStateDipoleMomentProjection_y_m = np.dot(WideAxisUnit_m,GroundStateDipoleMoment_m)
GroundStateDipoleMomentProjection_z_m = np.dot(VerticalAxisUnit_m,GroundStateDipoleMoment_m)

ExcitedStateDipoleMomentProjection_x_m = np.dot(LongAxisUnit_m,ExcitedStateDipoleMoment_m)
ExcitedStateDipoleMomentProjection_y_m = np.dot(WideAxisUnit_m,ExcitedStateDipoleMoment_m)
ExcitedStateDipoleMomentProjection_z_m = np.dot(VerticalAxisUnit_m,ExcitedStateDipoleMoment_m)

TransitionDipoleMomentProjection_x_m = np.dot(LongAxisUnit_m,TransitionDipoleMoment_m)
TransitionDipoleMomentProjection_y_m = np.dot(WideAxisUnit_m,TransitionDipoleMoment_m)
TransitionDipoleMomentProjection_z_m = np.dot(VerticalAxisUnit_m,TransitionDipoleMoment_m)



### Finding molecule n's dipole moments ###

GroundStateDipoleMomentProjection_x_n = GroundStateDipoleMomentProjection_x_m*LongAxisUnit_n[0] + GroundStateDipoleMomentProjection_y_m*WideAxisUnit_n[0] + GroundStateDipoleMomentProjection_z_m*VerticalAxisUnit_n[0]
GroundStateDipoleMomentProjection_y_n = GroundStateDipoleMomentProjection_x_m*LongAxisUnit_n[1] + GroundStateDipoleMomentProjection_y_m*WideAxisUnit_n[1] + GroundStateDipoleMomentProjection_z_m*VerticalAxisUnit_n[1]
GroundStateDipoleMomentProjection_z_n = GroundStateDipoleMomentProjection_x_m*LongAxisUnit_n[2] + GroundStateDipoleMomentProjection_y_m*WideAxisUnit_n[2] + GroundStateDipoleMomentProjection_z_m*VerticalAxisUnit_n[2]

GroundStateDipoleMoment_n = np.array([GroundStateDipoleMomentProjection_x_n,GroundStateDipoleMomentProjection_y_n,GroundStateDipoleMomentProjection_z_n])
GroundStateDipoleMomentLength_n = np.linalg.norm(GroundStateDipoleMoment_n)

ExcitedStateDipoleMomentProjection_x_n = ExcitedStateDipoleMomentProjection_x_m*LongAxisUnit_n[0] + ExcitedStateDipoleMomentProjection_y_m*WideAxisUnit_n[0] + ExcitedStateDipoleMomentProjection_z_m*VerticalAxisUnit_n[0]
ExcitedStateDipoleMomentProjection_y_n = ExcitedStateDipoleMomentProjection_x_m*LongAxisUnit_n[1] + ExcitedStateDipoleMomentProjection_y_m*WideAxisUnit_n[1] + ExcitedStateDipoleMomentProjection_z_m*VerticalAxisUnit_n[1]
ExcitedStateDipoleMomentProjection_z_n = ExcitedStateDipoleMomentProjection_x_m*LongAxisUnit_n[2] + ExcitedStateDipoleMomentProjection_y_m*WideAxisUnit_n[2] + ExcitedStateDipoleMomentProjection_z_m*VerticalAxisUnit_n[2]

ExcitedStateDipoleMoment_n = np.array([ExcitedStateDipoleMomentProjection_x_n,ExcitedStateDipoleMomentProjection_y_n,ExcitedStateDipoleMomentProjection_z_n])
ExcitedStateDipoleMomentLength_n = np.linalg.norm(ExcitedStateDipoleMoment_n)

TransitionDipoleMomentProjection_x_n = TransitionDipoleMomentProjection_x_m*LongAxisUnit_n[0] + TransitionDipoleMomentProjection_y_m*WideAxisUnit_n[0] + TransitionDipoleMomentProjection_z_m*VerticalAxisUnit_n[0]
TransitionDipoleMomentProjection_y_n = TransitionDipoleMomentProjection_x_m*LongAxisUnit_n[1] + TransitionDipoleMomentProjection_y_m*WideAxisUnit_n[1] + TransitionDipoleMomentProjection_z_m*VerticalAxisUnit_n[1]
TransitionDipoleMomentProjection_z_n = TransitionDipoleMomentProjection_x_m*LongAxisUnit_n[2] + TransitionDipoleMomentProjection_y_m*WideAxisUnit_n[2] + TransitionDipoleMomentProjection_z_m*VerticalAxisUnit_n[2]

TransitionDipoleMoment_n = np.array([TransitionDipoleMomentProjection_x_n,TransitionDipoleMomentProjection_y_n,TransitionDipoleMomentProjection_z_n])
TransitionDipoleMomentLength_n = np.linalg.norm(TransitionDipoleMoment_n)

StaticDipoleDifference_m = np.subtract(ExcitedStateDipoleMoment_m,GroundStateDipoleMoment_m)
StaticDipoleDifferenceLength_m = np.linalg.norm(StaticDipoleDifference_m)
StaticDipoleDifference_n = np.subtract(ExcitedStateDipoleMoment_n,GroundStateDipoleMoment_n)
StaticDipoleDifferenceLength_n = np.linalg.norm(StaticDipoleDifference_n)



### Printing out dipole vector lengths and verifying values ###

print("\nMagnitude of ground state dipole vector for molecule m =", GroundStateDipoleMomentLength_m)
print("Magnitude of ground state dipole vector for molecule n =", GroundStateDipoleMomentLength_n)

print("\nMagnitude of excited state dipole vector for molecule m =", ExcitedStateDipoleMomentLength_m)
print("Magnitude of excited state dipole vector for molecule n =", ExcitedStateDipoleMomentLength_n)

print("\nMagnitude of transition dipole vector for molecule m =", TransitionDipoleMomentLength_m)
print("Magnitude of transition dipole vector for molecule n =", TransitionDipoleMomentLength_n)

print("\nMagnitude of static dipole difference vector for molecule m =", StaticDipoleDifferenceLength_m)
print("Magnitude of static dipole difference vector for molecule n =", StaticDipoleDifferenceLength_n , "\n")


if abs(GroundStateDipoleMomentLength_m - GroundStateDipoleMomentLength_n) > .00000001:
    warning = warning + 1
    dipolewarning = dipolewarning + 1
if abs(ExcitedStateDipoleMomentLength_m - ExcitedStateDipoleMomentLength_n) > .00000001:
    dipolewarning = dipolewarning + 1    
    warning = warning + 1
if abs(TransitionDipoleMomentLength_m - TransitionDipoleMomentLength_n) > .00000001:
    dipolewarning = dipolewarning + 1    
    warning = warning + 1

if dipolewarning != 0:
    print("WARNING! One or more of the dipole magnitudes for molecules m and n are different. \nCoordinate systems might not be defined well.")
    
if warning == 0:
    print("\nDipole vector magnitudes are equal, the coordinate systems are OK.")


### Calculating J and K ###

print("\n\n~~~~~~~~~~ RESULTS ~~~~~~~~~~\n")

DotProductStaticDipoleDifferenceVectors = np.dot(StaticDipoleDifference_m,StaticDipoleDifference_n)
DotProductStaticDipoleDifferenceSeparation_m = np.dot(StaticDipoleDifference_m,DyeSeparationVector)
DotProductStaticDipoleDifferenceSeparation_n = np.dot(StaticDipoleDifference_n,DyeSeparationVector)
DotProductTranistionDipoleVectors = np.dot(TransitionDipoleMoment_m,TransitionDipoleMoment_n)
DotProductTranistionDipoleSeparation_m = np.dot(TransitionDipoleMoment_m,DyeSeparationVector)
DotProductTranistionDipoleSeparation_n = np.dot(TransitionDipoleMoment_n,DyeSeparationVector)

J = (1/(4*3.1415*Permittivity*0.000000000008845))*((DotProductTranistionDipoleVectors/(DyeSeparationLength**3))-(3*DotProductTranistionDipoleSeparation_m*DotProductTranistionDipoleSeparation_n/(DyeSeparationLength**5)))
K = (1/(4*3.1415*Permittivity*0.000000000008845))*((DotProductStaticDipoleDifferenceVectors/(DyeSeparationLength**3))-(3*DotProductStaticDipoleDifferenceSeparation_m*DotProductStaticDipoleDifferenceSeparation_n/(DyeSeparationLength**5)))

JNoPreFactor = ((DotProductTranistionDipoleVectors/(DyeSeparationLength**3))-(3*DotProductTranistionDipoleSeparation_m*DotProductTranistionDipoleSeparation_n/(DyeSeparationLength**5)))
KNoPreFactor = ((DotProductStaticDipoleDifferenceVectors/(DyeSeparationLength**3))-(3*DotProductStaticDipoleDifferenceSeparation_m*DotProductStaticDipoleDifferenceSeparation_n/(DyeSeparationLength**5)))

print("J =", J)
print("K =", K)

print("\nJ with no prefactor =", JNoPreFactor)
print("K with no prefactor =", KNoPreFactor)

if warning !=0:
    print("\n!!! Calculated J and K values are probably not accurate, check coordinate descriptions. !!!")

Dye Separation = 14.708829491159385 Angstroms

~~~~~ Corrections are being used for the initial coordinate axes guesses! ~~~~~

Magnitude of ground state dipole vector for molecule m = 72.17643509498097
Magnitude of ground state dipole vector for molecule n = 72.17643509498096

Magnitude of excited state dipole vector for molecule m = 72.17992747114671
Magnitude of excited state dipole vector for molecule n = 72.1799274711467

Magnitude of transition dipole vector for molecule m = 37.360426907544834
Magnitude of transition dipole vector for molecule n = 37.36042690754484

Magnitude of static dipole difference vector for molecule m = 0.06590614538872898
Magnitude of static dipole difference vector for molecule n = 0.06590614538873413 


Dipole vector magnitudes are equal, the coordinate systems are OK.


~~~~~~~~~~ RESULTS ~~~~~~~~~~

J = -1523065123.716088
K = -17053.459672986864

J with no prefactor = -0.16928300746813174
K with no prefactor = -1.8954284332479098e-06
